In [1]:
#!pip install lightfm

In [2]:
import pandas as pd
import numpy as np
import datetime
from random import sample

from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import auc_score

/Users/nayoung/opt/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [44]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

SPOTIPY_CLIENT_ID='608e3b1bcba14f8bb1ed22fa44291d13'
SPOTIPY_CLIENT_SECRET='848e0d68cf6a4d00b07f583a69aa35dd'

#sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

URI = 'http://localhost:8080'

scope = " ".join(['playlist-modify-public',"user-top-read","user-read-recently-played","playlist-read-private"])

username = 'skdud712'

try:
    token = spotipy.util.prompt_for_user_token(client_id = SPOTIPY_CLIENT_ID,
                                               client_secret = SPOTIPY_CLIENT_SECRET,
                                               redirect_uri = URI,
                                               scope = scope,
                                               username=username)
except:
    os.remove(f'cache-{username}')
    token = spotipy.util.prompt_for_user_token(username=username)
    
spotify = spotipy.Spotify(auth=token)

In [4]:
playlists = spotify.current_user_playlists()['items']
playlist_id = ''
for i in playlists:
    if i['name'] == 'capstone':
        playlist_id = i['id']

In [49]:
seed_artists = []
seed_tracks = []

for i in spotify.current_user_top_tracks(limit=50, offset=0, time_range='medium_term')['items']:
    seed_artists += [track['artists'][0]['id']]
    seed_tracks.append(i['id'])
    #seed_genres.add(track['genre'])
seed_tracks

['4A6BplRJXLfp9GsDwzk3LV',
 '1TmvMsKakEUCtLgkGYEZp4',
 '214fNrcCdFO4BDDZhejst0',
 '5ipJi9h2ghaThn6EUwO3B2',
 '5rIsrawNyPv1q8v11iwJsh',
 '7rUURVpd1ucUhkbnYNlJbw',
 '0cbh6m6Ita9t4sKVDJQZFP',
 '1NfPDgcPWGJk7u9wFPEpRe',
 '1gpF8IwQQj8qOeVjHfIIDU',
 '1mdEvT7X3Rs7vL75fRb8oo',
 '1rhWvjfn9UuR5L3DYwWLNq',
 '1xEGxyC38Ms7uq5C5jlgXD',
 '2COGIRKgW0Ur14roVSJ0ZA',
 '2dtY3bZk61fcWvt4oumfnH',
 '2dyyOwe0KxTZTJjjXDeLmU',
 '2wdqUPeZegPUHshm1YIsGa',
 '34rzJNfTd5e9s3TICFMPD0',
 '3JpsE1X8RiHZer0Y32cmM6',
 '3iDE9gIZbt57anQvECuy6Y',
 '46xDHGYxW0Z2o20sOsWLAx',
 '5dzHlLBWdxdQySpvZLHcu9',
 '5qYp1Wf3gh0lGWdIS5lIrX',
 '6ejba4BypjeVSs5VfDW8w2',
 '6txRAxfSDogqUd6BYzfocG',
 '78cEmtHsaekYyw4Z12oBR2',
 '7FjZU7XFs7P9jHI9Z0yRhK',
 '7lszBgdvdJGixKJHPwXHDl',
 '7nKQ5WAcjnG48knyLuo8gO',
 '07Wqp9LcTwzdIyuI0twXjx',
 '0UEknUD00qJuDjljnI1zw9',
 '0lLDInOmkZTWz14Lm0dcMY',
 '0mwCLKphlE8pzj6HKCWOlU',
 '0qhTNlYrAJBXpHHzay7qvM',
 '12XPa1Q3JzQchPjmRxPjXg',
 '1EYSS9Lp7a9f20C3FaOVgI',
 '1N9TrsZZXX8GQvOvD3PV23',
 '1PQZCoS2ZRPZFSspJ1GbR6',
 

In [5]:
seed_artists = []
seed_tracks = []
#seed_genres = set()
for i in spotify.playlist_tracks(playlist_id=playlist_id)['items']:
    track = i['track']
    seed_artists += [track['artists'][0]['id']]
    seed_tracks.append(track['id'])
    #seed_genres.add(track['genre'])

In [6]:
# stuff = pd.read_csv('https://query.data.world/s/go22golrhaeqllglpuxnnd7irb3l2j')
# stuff['WeekID'] = pd.to_datetime(stuff['WeekID'])
# stuff.head()

In [7]:
features = pd.read_excel('https://query.data.world/s/2hymngpmogoje5bwt3ikufdxjdgkn3')
features = features.dropna(subset=['spotify_track_id', 'spotify_genre']).drop_duplicates(subset='spotify_track_id').drop_duplicates(subset='spotify_track_id')#[features.columns[0:5]]
features['spotify_genre'] = [x.strip('[]').strip('\'').strip('"').split('\', \'') for x in features['spotify_genre']]
features.head()

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,[pop reggaeton],3jbT1Y5MoPwEIpZndDDwVq,NaN,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,...,10.0,-4.803,0.0,0.0735,0.01700,0.000016,0.1790,0.623,80.002,4.0
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"[dance pop, pop]",2VxeLyX666F8uXCJ0dZF8B,NaN,A Star Is Born Soundtrack,0.0,215733.0,88.0,...,7.0,-6.362,1.0,0.0308,0.37100,0.000000,0.2310,0.323,95.799,4.0
3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,"[dfw rap, melodic rap, rap]",0Xek5rqai2jcOWCYWJfVCF,NaN,Hollywood's Bleeding,1.0,196760.0,86.0,...,6.0,-4.169,1.0,0.2100,0.05880,0.000000,0.0955,0.667,76.388,4.0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"[dfw rap, rap, southern hip hop, trap]",2biNa12dMbHJrHVFRt8JyO,https://p.scdn.co/mp3-preview/fa6fa6f6f363be29...,Bacc At It Again,1.0,228185.0,61.0,...,8.0,-5.725,0.0,0.1680,0.00124,0.000001,0.0716,0.856,135.979,4.0
5,The ArcherTaylor Swift,Taylor Swift,The Archer,"[dance pop, pop, post-teen pop]",3pHkh7d0lzM2AldUtz2x37,NaN,Lover,0.0,211240.0,76.0,...,0.0,-9.375,1.0,0.0401,0.12000,0.005690,0.0663,0.166,124.344,4.0


In [8]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

f = features.join(pd.DataFrame.sparse.from_spmatrix(
    mlb.fit_transform(features.pop('spotify_genre')),
    index=features.index,
    columns=mlb.classes_))
f.head()

,SongID,Performer,Song,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,world worship,worship,wrestling,wu fam,xhosa,yacht rock,ye ye,yodeling,yugoslav rock,zolo
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,3jbT1Y5MoPwEIpZndDDwVq,NaN,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,0.734,...,0,0,0,0,0,0,0,0,0,0
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,2VxeLyX666F8uXCJ0dZF8B,NaN,A Star Is Born Soundtrack,0.0,215733.0,88.0,0.572,...,0,0,0,0,0,0,0,0,0,0
3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,0Xek5rqai2jcOWCYWJfVCF,NaN,Hollywood's Bleeding,1.0,196760.0,86.0,0.542,...,0,0,0,0,0,0,0,0,0,0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,2biNa12dMbHJrHVFRt8JyO,https://p.scdn.co/mp3-preview/fa6fa6f6f363be29...,Bacc At It Again,1.0,228185.0,61.0,0.948,...,0,0,0,0,0,0,0,0,0,0
5,The ArcherTaylor Swift,Taylor Swift,The Archer,3pHkh7d0lzM2AldUtz2x37,NaN,Lover,0.0,211240.0,76.0,0.292,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28487,"Tonight, TonightThe Smashing Pumpkins",The Smashing Pumpkins,"Tonight, Tonight",7bu0znpSbTks0O6I98ij0W,NaN,Mellon Collie And The Infinite Sadness (Deluxe...,0.0,254626.0,69.0,0.287,...,0,0,0,0,0,0,0,0,0,0
28488,Tonight's The Night (Gonna Be Alright)Rod Stewart,Rod Stewart,Tonight's The Night (Gonna Be Alright),10LRRKPmx8uHcSxLVWK0ct,https://p.scdn.co/mp3-preview/de4772519fbe8134...,A Night on the Town (Deluxe Edition),0.0,237120.0,55.0,0.550,...,0,0,0,0,0,0,0,0,0,0
28489,Tonight's The NightBLACKstreet,BLACKstreet,Tonight's The Night,3scEwKiAkfdPUdZ6k1AEoW,NaN,Blackstreet,0.0,258026.0,32.0,0.585,...,0,0,0,0,0,0,0,0,0,0
28490,Tonight's The NightS.S.O.,S.S.O.,Tonight's The Night,5Gxa3zKb3xoS9G0y4fJnP1,https://p.scdn.co/mp3-preview/fb63477c965b182b...,Funky Chicken: Belgian Grooves From The 70's,0.0,170506.0,13.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#f1[f1.columns[21:]]

In [10]:
#print(f1.columns[21:])

In [11]:
a = f[f.columns[21:]].sum()
#a

In [12]:
#a.sort_values(ascending=True).plot.hist(bins=3000,ylim=(0,50),xlim=(0,200))#,  xlim=(0,10), ylim=(0,500))#  xlim=(0,1000))

In [13]:
a.mean(), a.median()

(114.31170108161258, 6.0)

In [29]:
tt_split = int(len(seed_tracks)*0.8)
seed_train = sample(seed_tracks, tt_split)
seed_test = seed - set(seed_train)
len(seed_train), len(seed_test)
#seed['feedback'] = 1
#seed.head()

(24, 6)

In [ ]:
#a['']

In [ ]:
#len(a[a<114].index), len(a[a>=114].index)

In [34]:
for i in range(0,5):
    tt_split = int(len(seed_tracks)*0.8)
    seed_train = sample(seed_tracks, tt_split)
    seed_test = seed - set(seed_train)
    print('Test # %s', i)
    
    for limit in range(0, 200, 20):
        #limit = 114
        #len(a[a<limit].index), len(a[a>=limit].index)
        rare_genre = a[a<limit].index.to_list()+['']
        f1 = f.drop(columns=rare_genre)
        features_list = f1.columns[9:].to_list()
        f2 = f1[['spotify_track_id']+features_list].dropna(how='any')
        f3 = f2#[~f2[f2.columns[21:].to_list()].eq(0).all(1)]

        item_features = []
        for index, row in f3.iterrows():
            f_dict = {}
            for f_col in features_list:
                f_dict.update({str(f_col):row[f_col]})
            item_features.append((row['spotify_track_id'],f_dict))

        #lfm implementation
        lfm_data = Dataset()
        lfm_data.fit(users=[username],items=f3['spotify_track_id'], item_features=features_list)

        item_features_built = lfm_data.build_item_features(item_features)
        #item_features_built


        train = f1.loc[f1['spotify_track_id'].isin(seed_train)]
        test = f1.loc[f1['spotify_track_id'].isin(seed_test)]
        train_interactions, train_w = lfm_data.build_interactions([(username, x) for x in train['spotify_track_id'].to_list()])
        test_interactions, test_w = lfm_data.build_interactions([(username, x) for x in test['spotify_track_id'].to_list()])

        model = LightFM(loss='bpr')
        model.fit(interactions=train_interactions, item_features= item_features_built, epochs=30, num_threads=2)
        n_users, n_items = train_interactions.shape # no of users * no of items
        user_id_map, user_feature_map, item_id_map, item_feature_map = lfm_data.mapping()
        model.predict(user_id_map[username], np.arange(n_items)) # means predict for all

        print(limit)
        train_auc = auc_score(model,
                          train_interactions,
                          item_features= item_features_built
                         ).mean()
        print('Hybrid training set AUC: %s' % train_auc)
        test_auc = auc_score(model,
                              test_interactions,
                              item_features= item_features_built
                             ).mean()
        print('Hybrid test set AUC: %s' % test_auc)

Test # %s 0
0
Hybrid training set AUC: 0.52874106
Hybrid test set AUC: 0.5130351
20
Hybrid training set AUC: 0.52988774
Hybrid test set AUC: 0.5516055
40
Hybrid training set AUC: 0.5421596
Hybrid test set AUC: 0.5454377
60
Hybrid training set AUC: 0.43386093
Hybrid test set AUC: 0.3123385
80
Hybrid training set AUC: 0.5068405
Hybrid test set AUC: 0.533359
100
Hybrid training set AUC: 0.50012326
Hybrid test set AUC: 0.5107366
120
Hybrid training set AUC: 0.54109687
Hybrid test set AUC: 0.53896993
140
Hybrid training set AUC: 0.43074062
Hybrid test set AUC: 0.282613
160
Hybrid training set AUC: 0.5299556
Hybrid test set AUC: 0.5497852
180
Hybrid training set AUC: 0.4088955
Hybrid test set AUC: 0.3369098
Test # %s 1
0
Hybrid training set AUC: 0.52255064
Hybrid test set AUC: 0.52248687
20
Hybrid training set AUC: 0.5221773
Hybrid test set AUC: 0.46732625
40
Hybrid training set AUC: 0.4682013
Hybrid test set AUC: 0.2342128
60
Hybrid training set AUC: 0.5224881
Hybrid test set AUC: 0.4831527

In [50]:
rare_genre = a#[a<limit].index.to_list()+['']
f1 = features#.drop(columns=rare_genre)
features_list = f1.columns[9:].to_list()
f2 = f1[['spotify_track_id']+features_list].dropna(how='any')
f3 = f2#[~f2[f2.columns[21:].to_list()].eq(0).all(1)]

item_features = []
for index, row in f3.iterrows():
    f_dict = {}
    for f_col in features_list:
        f_dict.update({str(f_col):row[f_col]})
    item_features.append((row['spotify_track_id'],f_dict))

#lfm implementation
lfm_data = Dataset()
lfm_data.fit(users=[username],items=f3['spotify_track_id'], item_features=features_list)

item_features_built = lfm_data.build_item_features(item_features)
#item_features_built

for i in range(0,5):
    tt_split = int(len(seed_tracks)*0.8)
    seed_train = sample(seed_tracks, tt_split)
    seed_test = seed - set(seed_train)
    print('Test # %s' % i)

    train = f1.loc[f1['spotify_track_id'].isin(seed_train)]
    test = f1.loc[f1['spotify_track_id'].isin(seed_test)]
    train_interactions, train_w = lfm_data.build_interactions([(username, x) for x in train['spotify_track_id'].to_list()])
    test_interactions, test_w = lfm_data.build_interactions([(username, x) for x in test['spotify_track_id'].to_list()])

    model = LightFM(loss='bpr')
    model.fit(interactions=train_interactions, item_features= item_features_built, epochs=30, num_threads=2)
    n_users, n_items = train_interactions.shape # no of users * no of items
    user_id_map, user_feature_map, item_id_map, item_feature_map = lfm_data.mapping()
    model.predict(user_id_map[username], np.arange(n_items)) # means predict for all

    train_auc = auc_score(model,
                          train_interactions,
                          item_features= item_features_built).mean()
    print('Hybrid training set AUC: %s' % train_auc)
    test_auc = auc_score(model,
                         test_interactions,
                         item_features= item_features_built).mean()
    print('Hybrid test set AUC: %s' % test_auc)

Test # 0
Hybrid training set AUC: nan
Hybrid test set AUC: 0.4458203
Test # 1
Hybrid training set AUC: nan
Hybrid test set AUC: 0.40309563
Test # 2
Hybrid training set AUC: nan
Hybrid test set AUC: 0.5638467
Test # 3
Hybrid training set AUC: nan
Hybrid test set AUC: 0.45991936
Test # 4
Hybrid training set AUC: nan
Hybrid test set AUC: 0.62274426


<ipython-input-50-91a308c42a55>:38: RuntimeWarning: Mean of empty slice.
  train_auc = auc_score(model,
/Users/nayoung/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
limit = 114
len(a[a<limit].index), len(a[a>=limit].index)

In [ ]:
rare_genre = a[a<limit].index.to_list()+['']
f1 = f.drop(columns=rare_genre)
f1

In [ ]:
#f1.columns[21:]

In [ ]:
# f1.danceability.plot.hist()

In [ ]:
# f1.energy.plot.hist()

In [ ]:
features_list = f1.columns[9:].to_list()
#features_list

In [ ]:
f2 = f1[['spotify_track_id']+features_list].dropna(how='any')
f3 = f2#[~f2[f2.columns[21:].to_list()].eq(0).all(1)]
f3

In [ ]:
item_features = []
for index, row in f3.iterrows():
    f_dict = {}
    for f_col in features_list:
        f_dict.update({str(f_col):row[f_col]})
    item_features.append((row['spotify_track_id'],f_dict))

In [ ]:
#len(item_features), item_features[0]

In [ ]:
lfm_data = Dataset()
lfm_data.fit(users=[username],items=f3['spotify_track_id'], item_features=features_list)

In [ ]:
user_id_map, user_feature_map, item_id_map, item_feature_map = lfm_data.mapping()
#user_id_map, user_feature_map

In [ ]:
#item_id_map

In [ ]:
#item_feature_map

In [ ]:
item_features_built = lfm_data.build_item_features(item_features)
item_features_built

In [ ]:
seed = f1.loc[f1['spotify_track_id'].isin(seed_tracks)]
#seed['feedback'] = 1
seed.head()

In [ ]:
tt_split = int(len(seed)*0.9)
train = seed[:tt_split]
test = seed[tt_split:]
train_interactions, train_w = lfm_data.build_interactions([(username, x) for x in train['spotify_track_id'].to_list()])
test_interactions, test_w = lfm_data.build_interactions([(username, x) for x in test['spotify_track_id'].to_list()])

In [ ]:
#train_interactions

In [ ]:
model = LightFM(loss='bpr')
model.fit(interactions=train_interactions, item_features= item_features_built, epochs=30, num_threads=2)

In [ ]:
n_users, n_items = train_interactions.shape # no of users * no of items
model.predict(user_id_map[username], np.arange(n_items)) # means predict for all

In [ ]:
train_auc = auc_score(model,
                      train_interactions,
                      item_features= item_features_built
                     ).mean()
print('Hybrid training set AUC: %s' % train_auc)

In [ ]:
test_auc = auc_score(model,
                      test_interactions,
                      item_features= item_features_built
                     ).mean()
print('Hybrid test set AUC: %s' % test_auc)

100
Hybrid training set AUC: 0.50012326
Hybrid test set AUC: 0.5107366
100
Hybrid training set AUC: 0.51013225
Hybrid test set AUC: 0.43036935
100
Hybrid training set AUC: 0.4520876
Hybrid test set AUC: 0.50326234
100
Hybrid training set AUC: 0.5107984
Hybrid test set AUC: 0.5957582
100
Hybrid training set AUC: 0.5750372
Hybrid test set AUC: 0.41212294
0.508
0.49




120
Hybrid training set AUC: 0.54109687
Hybrid test set AUC: 0.53896993
120
Hybrid training set AUC: 0.5533957
Hybrid test set AUC: 0.5812239
120
Hybrid training set AUC: 0.6102044
Hybrid test set AUC: 0.24758357
120
Hybrid training set AUC: 0.5189267
Hybrid test set AUC: 0.52129465
120
Hybrid training set AUC: 0.5489734
Hybrid test set AUC: 0.38209048
training 0.554
test 0.4542



160
Hybrid training set AUC: 0.5299556
Hybrid training set AUC: 0.5233223
Hybrid training set AUC: 0.3915281
Hybrid training set AUC: 0.4962457
Hybrid training set AUC: 0.56108457
0.5004

Hybrid test set AUC: 0.5497852
Hybrid test set AUC: 0.4840736
Hybrid test set AUC: 0.7488899
Hybrid test set AUC: 0.5969218
Hybrid test set AUC: 0.45166406
0.566